# **Capstone Project Notebook**

#### This is the final capstone project of **Enrico Pizzutilo**
It will be mainly used for the capstone project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Second Assignement


In [1]:
!pip install bs4

     |████████████████████████████████| 122kB 1.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050."
html_data = requests.get(url).text

In [7]:
soup = BeautifulSoup(html_data, "html5lib")
soup.title

<title>List of postal codes of Canada: M - Wikipedia</title>

In [56]:
#Used to check the tables present in soup --> leave # to compute them
#soup.find_all("tbody")
a=soup.find("tbody").find("tr")
print(a)
#for element in a.next_siblings:
#    print("-----------")
#    print(element)
#soup.find_all("td")

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>


<generator object next_siblings at 0x7f4a252cb360>

#### Create the Dataframe
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [142]:
neighborhood_df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
print(neighborhood_df)

for row in soup.find("tbody").find_all("tr"):
    col = row.find_all("td")
    if col != []:
        postcode = col[0].text
        borough = col[1].text
        neighborhood = col[2].text
        #print("{}, {}, {}".format(postcode, borough, neighborhood))
        neighborhood_df = neighborhood_df.append({'PostalCode':postcode, 'Borough':borough, 'Neighborhood':neighborhood}, ignore_index=True)

neighborhood_df

Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
...,...,...,...
282,M8Z,Etobicoke,Mimico NW\n
283,M8Z,Etobicoke,The Queensway West\n
284,M8Z,Etobicoke,Royal York South West\n
285,M8Z,Etobicoke,South of Bloor\n


- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [150]:
#drop the row with borough not assigned
neighborhood_df=neighborhood_df.drop(neighborhood_df[neighborhood_df['Borough']=='Not assigned'].index)
neighborhood_df
#neighborhood_df['Borough']=='Not assigned'

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M6A,North York,Lawrence Heights\n
6,M6A,North York,Lawrence Manor\n
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West\n
282,M8Z,Etobicoke,Mimico NW\n
283,M8Z,Etobicoke,The Queensway West\n
284,M8Z,Etobicoke,Royal York South West\n


- If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [176]:
#check if there is any non assigned neighborhood
a = neighborhood_df[neighborhood_df['Neighborhood']=='Not assigned']
a

,PostalCode,Borough,Neighborhood


- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [144]:
#testing the codes for the following loops
unique_postalcode_df = neighborhood_df[neighborhood_df['PostalCode']=='M8Z']
print(unique_postalcode_df)
neigh_list_same_pc = unique_postalcode_df['Neighborhood'].tolist()
neighborhoods = ''.join(neigh_list_same_pc).replace("\n", " , ")[0:-3]
borough = unique_postalcode_df['Borough'][unique_postalcode_df.index[0]]
print(borough, neighborhoods)

    PostalCode    Borough                Neighborhood
281        M8Z  Etobicoke  Kingsway Park South West\n
282        M8Z  Etobicoke                 Mimico NW\n
283        M8Z  Etobicoke        The Queensway West\n
284        M8Z  Etobicoke     Royal York South West\n
285        M8Z  Etobicoke            South of Bloor\n
Etobicoke Kingsway Park South West , Mimico NW , The Queensway West , Royal York South West , South of Bloor


In [170]:
#starting from the previous dataframe use a loop to have all the neighborhood with the same postal code in the same row

unique_postalcode_total_table = []
print(unique_postalcode_total_table)

for unique_postalcode in pd.unique(neighborhood_df["PostalCode"]):
    cell = {}
    unique_postalcode_df = neighborhood_df[neighborhood_df['PostalCode']==unique_postalcode]
    
    cell['PostalCode'] = unique_postalcode
    cell['Borough'] = unique_postalcode_df['Borough'][unique_postalcode_df.index[0]]
    cell['Neighborhood'] = ''.join(unique_postalcode_df['Neighborhood'].tolist()).replace("\n", " , ")[0:-3]
    
    #print(cell)
    unique_postalcode_total_table.append(cell)
    #print(unique_postalcode_total_table)
    
unique_postalcode_total_df = pd.DataFrame(unique_postalcode_total_table)
unique_postalcode_total_df

[]


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights , Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay , King's Mill Park , Kingsway Park ..."


- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [ ]:
unique_postalcode_total_d